### Real-Time Content Moderation System
This notebook presents a real-time content moderation pipeline that integrates toxicity detection and fact verification models. It acts as a safeguard before LLM-generated content is shown to users.

### Step 1: Install Required Libraries
Install the required libraries before proceeding:

In [ ]:
! pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

### Step 2: Import Libraries

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

### Step 3: Define the Safety Ensemble Class
This class loads both toxicity detection and fact-checking models and provides a method to analyze text.

In [ ]:
class SafetyEnsemble:
    def __init__(self, hf_token=None):
        """
        Initialize safety models with optional Hugging Face token
        """
        # Load toxicity detection model
        self.toxicity_model = AutoModelForSequenceClassification.from_pretrained(
            "facebook/roberta-hate-speech-dynabench-r4-target",
            use_auth_token=hf_token
        )

        # Load fact verification model (natural language inference)
        self.factcheck_model = AutoModelForSequenceClassification.from_pretrained(
            "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli",
            use_auth_token=hf_token
        )

        # Load a shared tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            "roberta-base",
            use_auth_token=hf_token
        )

    def analyze(self, text):
        """
        Analyze text for toxicity and factual accuracy
        """
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

        tox_score = self._get_toxicity_score(inputs)
        veracity, veracity_probs = self._get_veracity(inputs)

        return {
            "toxicity_risk": tox_score,
            "fact_accuracy": {
                "label": veracity,
                "probabilities": veracity_probs
            },
            "block": tox_score > 0.9 or veracity == "contradiction"
        }

    def _get_toxicity_score(self, inputs):
        """Return probability of toxicity (0 to 1)"""
        self.toxicity_model.eval()
        with torch.no_grad():
            outputs = self.toxicity_model(**inputs)
        # Handle binary or multi-class logits
        return torch.sigmoid(outputs.logits).item() if outputs.logits.shape[1] == 1 else torch.softmax(outputs.logits, dim=1)[0][1].item()

    def _get_veracity(self, inputs):
        """Use an NLI model to assess veracity"""
        self.factcheck_model.eval()
        with torch.no_grad():
            outputs = self.factcheck_model(**inputs)

        probs = torch.softmax(outputs.logits, dim=1)[0]
        label_idx = torch.argmax(probs).item()
        label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}

        return label_map[label_idx], [round(p.item(), 4) for p in probs]

### Step 4: Define a Dummy LLM for Testing

In [ ]:
class DummyLLM:
    """Mock LLM for demonstration purposes"""
    def generate(self, prompt: str):
        return {"generated_text": f"This is a safe response to: '{prompt}'."}


### Step 5: Run the Moderation Pipeline

In [ ]:
def main():
    safety = SafetyEnsemble()
    llm = DummyLLM()

    print("Safety Analysis Demo (type 'quit' to exit)")

    while True:
        prompt = input("\nEnter text to analyze: ").strip()
        if prompt.lower() in ('quit', 'exit'):
            break

        analysis = safety.analyze(prompt)

        print("\nAnalysis Results:")
        print(f"Toxicity Risk: {analysis['toxicity_risk']:.4f}")
        print(f"Fact Accuracy: {analysis['fact_accuracy']['label']}")
        print(f"  Probabilities: [E: {analysis['fact_accuracy']['probabilities'][0]:.2f}, "
              f"N: {analysis['fact_accuracy']['probabilities'][1]:.2f}, "
              f"C: {analysis['fact_accuracy']['probabilities'][2]:.2f}]")

        if analysis['block']:
            print("\n❌ Blocked - Reason:", end=" ")
            if analysis['toxicity_risk'] > 0.9:
                print("High toxicity risk", end="")
                if analysis['fact_accuracy']['label'] == "contradiction":
                    print(" and factual contradiction")
                else:
                    print()
            else:
                print("Factual contradiction")
        else:
            response = llm.generate(prompt)
            print(f"\n✅ Allowed - Generated response: {response['generated_text']}")


### Step 6: Run the Main Function

In [ ]:
if __name__ == "__main__":
    main()